### 6.1 Function Specification

#### 6.1.1 Function Definition

In contrast to statically typed languages, the input parameters and the return value are not explicitly typed; in fact, they don't even need to be declared. Even the function name is optional.

In [4]:
{[x] x*x}[3]
f:{[x] x*x}

9


#### 6.1.2 Function Notation and Terminology

The formal notation for function definition is,

{[p1;...;pn] e1; ...; em}

The optional p1, ... , pn are formal parameters. The expressions e1, ... , em – which presumably involve the parameters – represent the steps of an algorithm that are evaluated in order. **Note that while the expressions are sequenced left-to-right, an individual expression is always evaluated right-to-left.**

The semi-colons in the body are separators, not terminators. **Do not place a semi-colon after the last expression unless you deliberately want to suppress the return value.**


A **niladic** function in q is a function that has no meaningful input.
f:{[] … }
and is applied with empty argument – i.e., f[]. In this case, the function either returns a constant or has side effects, meaning that it accesses resources outside its body.

In [5]:
{[] 42} / pure function returns constant 42
{[] a*a} / impure function: references global a

{[] 42}


{[] a*a}


The maximum valence permitted as of this writing (Sep 2015) is 8; specifying more than eight parameters will cause an error. You can circumvent this restriction by encapsulating multiple parameters in a list or dictionary.

**The output value of a function written in functional programming style is the value of the last expression in its body.** This is the recommended style of writing q since it results in linear flow that is easy to follow and debug.

In [6]:
{[x;y] a:x*x; b:y*y; r:a+b; r}

{[x;y] a:x*x; b:y*y; r:a+b; r}


Q functions should be compact and modular: each function should perform a well-defined unit of work. Due to the power of q operators and built-in functions, many functions are one line.

**When a function exceeds 20 statements, you should look to refactor it.**`

### 6.1.3 Function Application

In [7]:
{[x] x*x}[3]

9


In [8]:
g:{[x;y] x+y}
g[3;4]

7


Function application in q is strict, meaning that expressions in arguments are evaluated before substitution is performed.

In [9]:
f:{[x] x*x}
f[0N!3+1]  / 0N prints output

16


In [10]:
const42:{[] 42}
const42[]
const42[3.4] / Apply a niladic function with no arguments or with an arbitrary argument, which is ignored

42


42


#### 6.1.4 Functions with No Return Value

Every q function has a return value but sometimes the value carries no meaningful information – i.e., it is used purely for side effects. For example, it may send off a message asynchronously without waiting for an acknowledgement of receipt or it may update a global table. To achieve the equivalent of a void return in C, make the expression after the last semi-colon in the function body empty.

In [11]:
fvoid:{[x] `a set x;}
fvoid 42        / no output,void
-3!fvoid 42     / In fact, the actual return value is the nil item :: that is a stand-in for void in this situation.

"::"


#### 6.1.6 Application of a Name

In [12]:
f:{x*x}
f[5]
`f[5]

25


25


This works because a global variable is implemented as a symbolic-name/function pair in a dictionary. Referring to the variable is actually lookup-by-name. Consequently application by name does not work for local variables or q system variables.

#### 6.1.7 Implicit Parameters

 Three implicit positional parameters x, y and z are automatically defined and available in a function’s expressions when no parameters are explicitly declared. Thus, the following two specifications result in equivalent input-output relations.

In [13]:
{[x;y;z] x+y-z}[1;4;5]
{x+y-z}[1;4;5]


0


0


#### 6.1.9 The Identity Function ::

When :: is used with function application syntax, it is the identity function – i.e., it returns its input as output.

In [14]:
::[42]
::[`a`b`c]

42


`a`b`c


In [15]:
/ :: 56  parse error

In [16]:
(::) 56

56


#### 6.1.10 Functions Are Data

Functions can be
- assigned to variables
- added to lists
- passed as arguments to another functions

In [17]:
apply:{x y}
apply[{x*x}; 5]

25


### 6.2 Call-by-Name

Ordinary function application in q uses **call-by-value, meaning that arguments are reduced to values at the start of application. In the process, copies are made so that any original values are undisturbed.** When the size of the input data is large, this copy may be prohibitive. In this case, there is an alternative method of application, call-by-name, in which names of (global) variables are passed instead of their values.

There is no special syntax for call-by-name in q. Instead, the function implementation simply expects a symbol containing the name of a variable and handles it appropriately. One example of a function that uses call-by-name is the built-in function get, which returns the value of the global variable whose name is passed.

In [18]:
a:42
/ get a - not working
get `a

42


The result of any built-in call-by-name function application is the symbol that was passed as input. This enables call-by-name functions to be chained – i.e., composed.

### 6.3 Local and Global Variables

#### 6.3.1 Defining Local and Global Variables

A variable assigned outside any function definition is called a **global variable**. Global variables are visible inside any function body.

In [19]:
a:42 / global
f:{[p1]; helper:{[p2] a*p2}; helper p1}
f 5

210


A variable that is assigned with : within a function body is called a **local variable**.

- A local variable cannot be used as an argument to a q function that uses call-by-name.
- A local variable is not visible within the body of a local function defined in the same scope. 

f:{[p1] x:42; helper:{[p2] x*p2}; helper p1} / x is local variable
/ f 5                                          / error as x is not visible

In [19]:
Instead, you must declare an additional parameter in the local functions and pass the local variable as an argument. 
Projection provides a relatively clean way to do this.

[0;31margument.[0m: [0;31margument.[0m

In [20]:
f:{[p1] x:42; helper:{[x; p2] x*p2}[x;]; helper p1}
f 5

210


#### 6.3.2 Assigning Global Variables within a Function

We recommend against :: for global assignment. Instead, use **set** which truly does global assignment and doesn't get hung up on a local-global name collision.

As we saw earlier, **set uses call-by-name**.

In [21]:
a:42
f:{a:98.6; `a set x}
f 43
a

`a


43


### 6.4 Projection

**Projection** means specifying only some of the parameters in function application, resulting in a function of the remaining parameters.

In [22]:
add:{x+y}          / function
add[42;]           / projection - also function
add[42]
add[42;][3]        / passing remaining arguments to  
add[;3][44]
add[42][3]

{x+y}[42;]


{x+y}[42]


45


47


45


Projection is related to **currying** in functional programming (named after Haskell Curry who did important early work on the theory of combinators). Both provide realizations of the mathematical observation that a function of two parameters is equivalent to a function of the first parameter that returns a function of the second parameter. This extends to multiple parameters as well. In q you simply project out successive parameters.

In [23]:
add:{x+y+z}
add[1;2;3]
add[1][2][3]

6


6


- We recommend that you do not omit trailing semi-colons in projections, as this can obscure the intent of code.

- The function body in a projection is captured at the time it is first encountered by the interpreter.
  Thus if the underlying function is subsequently changed, the projection is not.

#### 6.4.2 Operator Projection

When used in infix form, a q operator can be projected by fixing its left operand (only). This requires parentheses. For example, the projection of * onto its right operand is,

In [24]:
mul7:(7*)
mul7 5

35


In [25]:
-[;42] 98 / every operator is a function, so you can project it in prefix form.

56


#### 6.4.3 Multiple Projections

In [26]:
{x+y+z}[1;;3] 2
{x+y+z}[1;;][;3] 2
{x+y+z}[;;3][1;] 2

6


6


6


### 6.5 Everything Is a Map

Here we explore the deep relationship between q data structures and functions.

#### 6.5.1 Similarity of Notation
6.5.2 List of Indices, Keys and Arguments

notation for list indexing, dictionary lookup and monadic function application are identical

In [27]:
L:0 1 4 9 16 25 36
D:1 2 3!`a`b`c
f:{x*x}

L[1]
D[1]
f[1]

L 1
D 1
f 1

/List of Indices, Keys and Arguments
L 1 2
D 1 2
f 1 2


1


`a


1


1


`a


1


1 4


`a`b


1 4


- list is a map defined by positional retrieval via item indexing
- dictionary is a map defined by key-value lookup
- A function is a map defined by a sequence of expressions representing an algorithm for computing an output value from the input

In [28]:
m:(10 20 30; 100 200 300)
m[;2]

30 300


### 6.6 Atomic Functions

**The application of an atomic function is characterized by the fact that it recurses into an argument's structure until it gets to atoms and then applies there. It does this without explicit loops or other control flow constructs.**

#### 6.6.1 Monadic Atomic Functions and "map"

In [29]:
neg `a`b`c!(10 20; 30 40 50; 60)

a| -10 -20
b| -30 -40 -50
c| -60


A consequence of this behavior is that the result of a monadic atomic function always has the same shape as its argument – i.e., the output conforms to the input.

A special case of this is that applying an atomic function to a list is the same as applying it to each item in the list, which is sometimes described as "atomic functions extend automatically to lists." **Similar functionality is achieved with "foreach" in traditional languages and the higher-order function "map" in functional languages.**

#### 6.6.2 Dyadic Atomic Functions and zip

In [29]:
The arithmetic, comparison and relation operators are all atomic in both operands. This leads to four cases for application:

atom with atom
atom with list
list with atom
list with list
In the last case, the two list operands must have the same length.

[0;31mapplication[0m: [0;31mapplication[0m

#### 6.6.3 Creating Atomic Functions

Composition of atomic functions is atomic. Hence, the guaranteed way to build your own atomic functions is to compose built-in atomic functions. First monadic.

In [30]:
/ Monadic
f:{(x*x)+(2*x)-1}
f 1
f til 10

2


-1 2 7 14 23 34 47 62 79 98


In [31]:
/Dyadic
pyth:{sqrt (x*x)+y*y}
pyth [3;4]      / (3;4)
pyth [3; 4 5]   / (3;4) ; (3;5)
pyth [2 3; 5]   / (2;5) ; (3;5)
pyth [2 3; 4 5] / (2;4) ; (3;5)

5f


5 5.830952


5.385165 5.830952


4.472136 5.830952


### 6.7 Adverbs

**Adverbs are higher-order functions that modify the behavior of functions for application on lists. The terminology derives from thinking of q operators as verbs. **

Not every function is atomic, but with adverbs we may still  apply it to individual items of a data structure without loopy code. 

#### 6.7.1 Monadic each ( aka map)

count is not atomic as it collapses entire list. In particular, aggregate functions operate only at the top level of a nested list.

Suppose instead that we want to count each of the two items in our nested list. This is precisely what the each adverb does. **It modifies a monadic function so that it applies to each item in a list rather than to the list as a whole.** When used infix, each follows the function after (required) whitespace.

In [32]:
L:(1 4 5;2 1;3 0 9 8)
count each L      / infix form
each[count][L]    / prefix form

3 2 4


3 2 4


_**The higher-order function each is similar to "foreach" in imperative languages and is called "map" in functional programming languages.**_

We could say informally that each makes a non-atomic function act atomically… at least at one level. 
For deeply nested lists, you need to iterate each to apply a function at deeper levels.

In [33]:
L:((1 4 5; 7 8);(2 1; 3);(3 0 9 8; 9 9; 8 8))
count each L         / first level
(count each) each L  / second level

2 2 3


3 2
2 1
4 2 2


In [34]:
(count each) each (1 2 3; 10 20 30)

1 1 1
1 1 1


You can use each with any monadic function, although it is redundant on atomic functions.

It often arises that you want to convert a vector of length n to an n×1 matrix. You can do that with enlist each but flip enlist executes faster for large lists.

In [35]:
enlist 1001 1002 1004 1003      / enlist not monadic
enlist each 1001 1002 1004 1003 / enlists each element and returns list of results
flip enlist 1001 1002 1004 1003 / faster

1001 1002 1004 1003


1001
1002
1004
1003


1001
1002
1004
1003


#### 6.7.2 each-both ' ( aka zip)

The dyadic join operator , is not atomic since it consumes (copies of) both of its operands in their entirety when it concatenates.

In [36]:
L1:`ab`cd`ef
L2:`11`22`33
L1,L2 / join
L1,'L2    / join list items pair-wise - zip
,'[L1;L2] / prefix form

`ab`cd`ef`11`22`33


ab 11
cd 22
ef 33


ab 11
cd 22
ef 33


The adverb **each-both ' ** modifies a dyadic function (operator, verb) to apply pairwise to corresponding list items. Conventionally no whitespace should be between ' and the function it modifies but this does not generate an error. 
**In some programming languages, each-both is called "zip".**

The modified function also extends an atom to match a list. You will see this idiom often. For example,

In [37]:
1,'10 20 30
1 2 3,'10

1 10
1 20
1 30


1 10
2 10
3 10


In [38]:
2#'("abcde"; "fgh"; "ijklm") / take first 2 elements of each list
/ is equivalent to
((2#"abcde"); (2#"fgh"); (2#"ijklm"))

"ab"
"fg"
"ij"


"ab"
"fg"
"ij"


_The idiom ,' works fine for making a list of pairs from two simple lists but does not always work as expected with general lists._ For example it loses the enlisting of the singleton in the following.

In [39]:
L1:(enlist `a; `b)
L2:1 2
L1,'L2

`a 1
`b 2


A reliable way to make a list of pairs from a pair of lists is:

In [40]:
flip (L1;L2)

,`a 1
`b  2


A nice example of ,' arises when both operands are tables. Since a table is a list of records, it is possible to apply ,' to tables with the same number of rows. The record dictionaries in each row are upserted, resulting in a sideways join of the tables, as in the following example.

In [41]:
t1:([] c1:1 2 3)
t2:([] c2:`a`b`c)
t1,'t2

c1 c2
-----
1  a 
2  b 
3  c 


#### 6.7.3 each-left \:

An atomic operator extends an atom in the right operand to match a list on the left.

In [42]:
1 2 3+10

11 12 13


**The each-left adverb \: ** modifies a dyadic function so that it applies the second argument with each item of the first argument – for example, to append a string to each string in a list.

In [43]:
("abc"; "de"; enlist "f") ,\: ">"
/ equivalent to
("abc"; "de"; enlist "f") ,' (">",  ">",  ">")

"abc>"
"de>"
"f>"


"abc>"
"de>"
"f>"


#### 6.7.4 each-right /:

In [44]:
1+1 2 3

2 3 4


The each-right adverb /: modifies a dyadic function so that it applies the first argument to each item of the second argument. For example, to prepend a string to each string in a list.

In [45]:
"<" ,/: ("abc"; "de"; enlist "f")

"<abc"
"<de"
"<f"


In [46]:
/ equivalent to
("<",  "<",  "<"),'("abc"; "de"; enlist "f") 

"<abc"
"<de"
"<f"


In [47]:
"</",/:("abc"; "de"; enlist "f"),\:">"  / XML-like

"</abc>"
"</de>"
"</f>"


#### 6.7.5 Cross Product ( cross)

In [48]:
1 2 3,/:\:10 20
-3!1 2 3,/:\:10 20
raze 1 2 3,/:\:10 20 / raze - flattens one level
-3!raze 1 2 3,/:\:10 20

1 10 1 20
2 10 2 20
3 10 3 20


"((1 10;1 20);(2 10;2 20);(3 10;3 20))"


1 10
1 20
2 10
2 20
3 10
3 20


"(1 10;1 20;2 10;2 20;3 10;3 20)"


In [49]:
1 2 3 cross 10 20 / cross does the same

1 10
1 20
2 10
2 20
3 10
3 20


In [50]:
1 2 3,/:\:10 20
,/:\: [1 2 3;10 20]
raze ,/:\: [1 2 3;10 20]


1 10 1 20
2 10 2 20
3 10 3 20


1 10 1 20
2 10 2 20
3 10 3 20


1 10
1 20
2 10
2 20
3 10
3 20


#### 6.7.6 Over (/) for Accumulation  ( fold or recuce )

**The over adverb / ** is a higher-order function that provides the principle mechanism for recursion in q. In its simplest form it modifies a dyadic function **to accumulate results over a list.**

In [51]:
/ sum of numbers from 1 to 10
/ 0 - initial accumulator value
/ + - operation

0 +/ 1 2 3 4 5 6 7 8 9 10  / no whitespace b/w op and / as can be confused with comment, dyadic ( init val; list)
0 +/ 1+til 10
(+/) 1 2 3 4 5 6 7 8 9 10 / initial value can be skipped sometimes, () are mandatory, monadic ( list)
+/ [ 0; 1 2 3 4 5 6 7 8 9 10] / prefix form
+/ [ 1 2 3 4 5 6 7 8 9 10]    / prefix form

55


55


55


55


55


**In functional programming languages over is called fold or reduce. Along with each this completes the famous map-reduce paradigm.**

In [52]:
sum 1 2 3 4 5 6 7 8 9 10
(*/) 1 2 3 4 5 6 7 8 9 10 / product
prd 1 2 3 4 5 6 7 8 9 10 / product
(|/) 7 8 4 3 10 2 1 9 5 6 / maximum
max 7 8 4 3 10 2 1 9 5 6 / maximum
(&/) 7 8 4 3 10 2 1 9 5 6 / minimum
min 7 8 4 3 10 2 1 9 5 6 / minimum

55


3628800


3628800


10


10


1


1


Applying ,/ across a list concatenates the items, effectively eliminating the top level of nesting. If there is only one level of nesting, this flattens a list of lists to a plain list. **The function ,/ goes by the descriptive name raze.**

In [53]:
(,/)((1 2 3; 4 5); (100 200; 300 400 500))
raze ((1 2 3; 4 5); (100 200; 300 400 500))

1 2 3
4 5
100 200
300 400 500


1 2 3
4 5
100 200
300 400 500


You can use your own function with either form of over. The modified function can even be used infix, even though your original function cannot.

In [54]:
f:{2*x+y}
100 f/ 1 2 3 4 5 6 7 8 9 10
f/[100;1 2 3 4 5 6 7 8 9 10]

106472


106472


#### 6.7.7 More Over: Iteration ( / )

**Overload 1**

In this version of /, the left operand is a natural number indicating how many times to perform the iteration and the right operand is the initial value.

In [55]:
fib:{x,sum -2#x} / append sum of 2 last elements to sequence x
10 fib/ 1 1      / infix | execute function fib 10 times starting with sequence (1 1)
fib/[10; 1 1]    / prefix

1 1 2 3 5 8 13 21 34 55 89 144


1 1 2 3 5 8 13 21 34 55 89 144


**Overload 2**

Yet another version of / runs the recursion until convergence, or until a loop is detected. 
No additional parameters.

Approximation of square root.
square root of 2 is the root of equation -2+x*x=0
Inductive step: Given xn, the n+1st approximation is: xn – f(xn) / f'(xn)
Starting from any reasonable number ( e.g 10)

In [56]:
f:{-2+x*x}
secant:{[f;x;e] (f[x+e]-f[x-e])%2*e} / approximation of derivative
{x-f[x]%secant[f;x;1e-6]}/[1.5]      / overload of / runs until convergence
{x-f[x]%secant[f;x;1e-6]}\[10]      / changed over to scan

1.414214


10
5.1
2.746078
1.737195
1.444238
1.414526
1.414214
1.414214


** Overload 3**

The final overload of / is equivalent to a "while" loop in imperative programming. It provides a declarative way to specify a test to end the iteration. The iteration continues as long as the predicate result is 1b and stops otherwise. For example, we stop the computation of the Fibonacci sequence once it exceeds 1000 as follows.

In [57]:
fib:{x,sum -2#x}
fib/[{1000>last x}; 1 1] 
/ vs Overload 1
fib/[15; 1 1] 

1 1 2 3 5 8 13 21 34 55 89 144 233 377 610 987 1597


1 1 2 3 5 8 13 21 34 55 89 144 233 377 610 987 1597


#### 6.7.8 Scan \

**The scan adverb \ is a higher-order function that behaves just like / except that it returns all the intermediate accumulations instead of just the final one.** Whereas over produces an aggregate function that collapses a list, scan produces a function whose output has the same number of items as its input – i.e., a uniform function. Informally, scan is a "running" version of over.

In [58]:
0+/1 2 3 4 5 6 7 8 9 10 / over
0+\1 2 3 4 5 6 7 8 9 10 / scan

55


1 3 6 10 15 21 28 36 45 55


#### 6.7.9 each-previous (':)

**The adverb each-previous ': provides a declarative way to perform a dyadic operation on each item of a list with its predecessor.**
During traversal of the list, the current item is the left operand and the previous item is the right operand.

Since the initial item of the list does not have a predecessor, we must provide one in the left operand of the modified operator. One choice is the initial item of the source list. For example, to calculate the deltas of a list of prices.

In [59]:
100 -': 100 99 101 102 101
':[-][100; 100 99 101 102 101]
(-':)[100; 100 99 101 102 101]

0 -1 2 1 -1


0 -1 2 1 -1


0 -1 2 1 -1


Monadic form of (':) returns initial term of the input as first element

In [60]:
L:100 99 101 102 101
(-':)L
deltas L / returns first element - not good for anomalies
(%':)L
ratios L

100 -1 2 1 -1


100 -1 2 1 -1


100 0.99 1.020202 1.009901 0.9901961


100 0.99 1.020202 1.009901 0.9901961


In [61]:
sums L
(+\) L

100 199 300 402 503


100 199 300 402 503


In [62]:
sums deltas L
deltas sums L

100 99 101 102 101


100 99 101 102 101


To fix deltas:

In [63]:
deltas1:{first[x] -': x} / first deltas is 0
deltas1 L

0 -1 2 1 -1


Test whether successive terms match/don't match:

In [64]:
L1: 1 1 1 2 2 3 4 5 5 5 6 6
(~':) L1
not (~':) L1
differ L1  / differ is not (~':)

011010001101b


100101110010b


100101110010b


In [65]:
where differ L1
(where differ L1) cut L1

0 3 5 6 7 10


1 1 1
2 2
,3
,4
5 5 5
6 6


In [66]:
runs:(where differ L1) cut L1 / store runs
runs
ct:count each runs / store count of each run
ct
where ct=max ct
runs where ct=max ct / find the runs of maximum length

1 1 1
2 2
,3
,4
5 5 5
6 6


3 2 1 1 3 2


0 4


1 1 1
5 5 5


### 6.8 General Application

You won't be surprised that application of a function is actually a higher-order function that takes a function and value(s) and evaluates the function at the value(s).

#### 6.8.1 Verb @

Basic application comprises 
- retrieving a list item by index, 
- looking up a dictionary value by key, 
- or evaluating a monadic function ( one parameter only )

These are all instances of application of a univalent mathematical mapping.

In [71]:
/ infix
10 20 30 40@1 / list by index
10 20 30 40@1 3 / list by index
(`a`b`c!10 20 30)@`c`a / dictionary by key

20


20 40


30 10


In [73]:
/ prefix
@[10 20 30 40;1 3]
@[`a`b`c!10 20 30;`c`a]

20 40


30 10


In [74]:
/ function application
L:10 20 30 40
count L
count@L
@[count;L]

4


4


4


In [77]:
{x*x} L
{x*x}@L
@[{x*x};L]

100 400 900 1600


100 400 900 1600


100 400 900 1600


In [82]:
/ niladic
f:{4*6}
f[]
f@(::)
@[f;::] / nil argument
@[f;111] / any argument will suffice

24


24


24


24


#### 6.8.2 Verb Dot .

Application comprises 

- Indexing a list at depth, 
- retrieving a nested value from a dictionary and 
- evaluating a function with multiple parameters

These are all instances of application of a multi-variate mathematical mapping.

**The right argument of . must be a list.**

Separate verb . from its operands by whitespace if they are names or literal constants, as proximity to dots used in name spacing and in decimal numbers can lead to confusion.

In [84]:
L:(10 20 30; 40 50)
L[1][0]
L[1;0]
L . 1 0

40


40


40


In [86]:
d:`a`b`c!(10 20 30; 40 50; enlist 60)
d[`a][1]
d[`a;1]
d . (`a; 1)

20


20


20


In [ ]:
@ vs .
f@L -> f[L0]; f[L1]; f[L2]
f.L -> f[L0;L1;L2]

In [1]:
f:{x*x}
f@5
f . enlist 5
f . enlist (1 2 3)


25


25


1 4 9


#### 6.8.4 General Apply (@) with Monadic Functions

**applies to vector**

In [2]:
/ Retrieval:

L:10 20 30 40 50
@[L; 1]
@[L; 0 2]

20


10 30


Now instead of simply retrieving along the sub-domain, we supply a function to be applied along the sub-domain.
Only prefix notation

In [3]:
@[L; 1; neg]

10 -20 30 40 50


**Copy**: Effectively we reach inside (a copy of) the list, apply neg along the specified sub-domain, and return the modified list. Contrast this with normal application on the sub-domain, which returns only the modified items.

In [4]:
neg L@0 1

-10 -20


**The significance of enhanced application is that it returns the entire modified list**, which allows the result to be chained (i.e., composed) into further operations.

The syntax for general application of a monadic atomic function on a list is,

@[L;I;f]

For compound data structures such as nested lists, tables and keyed tables, application of @ occurs along a sub-domain at the top level. We'll see how to reach down into data structure in the next section.

In [5]:
m:(10 20 30; 100 200 300; 1000 2000 3000)
@[m; 0 2; neg]

-10   -20   -30  
100   200   300  
-1000 -2000 -3000


**In-place:**
All the examples we have shown thus far work against copies of the input data structure. It is also possible to modify the original data structure in place. Although q does not have references, general application provides pass-by-name with the name being a symbol. We demonstrate with a simple example. In the first application, the original L is unchanged; in the second it is modified.

In [11]:
L:10 20 30 40
@[L;0 2;neg]   / pass-by-value
L
@[`L; 0 2; neg] / pass-by-name
L

-10 20 -30 40


10 20 30 40


`L


-10 20 -30 40


Observe that the result of an application of pass-by-name is the passed symbolic name. This allows chaining of function applications that modify in place.

#### 6.8.5 General Apply (@) with Dyadic Functions

**applies to vector**

In [13]:
L:10 20 30 40
@[L; 0 1; +; 100 200] / 0 1 - indices; 100 200 - second arguments of dyadic func

100 200+L@0 1 / normal application

110 220 30 40


110 220


In [ ]:
Apply only on top-level.

A useful case of dyadic application uses the assignment operator : to modify values along the sub-domain.

In [14]:
L:10 20 30 40
@[L; 0 2; :; 42 43]

42 20 43 40


As in the monadic case, to apply to the original in place, instead of to a copy, use pass-by-name.

#### 6.8.6 General Apply (.) for Monadic Functions

**indexing-at-depth**

In [16]:
m:(10 20 30; 100 200 300)
d:`a`b`c!(10 20 30; 40 50; enlist 60)

In [17]:
.[m; 0 1 ; neg] / reaches to particular point in data structure and applies function(monadic)

10  -20 30 
100 200 300


In [18]:
.[d; (`a; 1); neg]

a| 10 -20 30
b| 40 50
c| ,60


In [ ]:
Works on copy of data, to modify in-place use symbol name (`m or `d)

We previously saw that to elide an index in . retrieval, we place the nil item :: in that slot. The same holds for general application, where it means apply the function along all indices at that level.

In [19]:
.[m; (0; ::) ; neg] / apply for entire first row (index 0)

-10 -20 -30
100 200 300


#### 6.8.7 General Apply (.) for Dyadic Functions

In [23]:
m:(10 20 30; 100 200 300)
.[m; 0 1 ; neg] / monadic
.[m; 0 1 ; +; 2] / dyadic - added 2 to element with indices [0;1]
.[m; (::; 1); +; 1 2]

10  -20 30 
100 200 300


10  22  30 
100 200 300


10  21  30 
100 202 300


In [25]:
d:`a`b`c!(100 200 300; 400 500; enlist 600)
d

a| 100 200 300
b| 400 500
c| ,600


In [26]:
.[`d; (::; 0); :; 42]

`d


In [27]:
d

a| 42 200 300
b| 42 500
c| ,42
